<a href="https://colab.research.google.com/github/AdityaLoth/Mask-Detection-using-Google-Colab/blob/master/mask_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FACE MASK DETECTION**

During this pandemic people are coming with various new and creative ideas to tackle Covid-19. One such idea is to detect whether a person is wearing mask or not at public places or at their workplaces. This has been made possible with the help of Deep Learning and Computer Vision.
This notebook took inspiration from one such [PROJECT](https://github.com/chandrikadeb7/Face-Mask-Detection) by Chandrika Deb.This is a Google colab implementation of Face Mask detection project. 


This notebook is divided into 4 parts:

1.   Setting up the environment
2.   Training the model
3.   Detection on Images
4.   Detection on Videos(using WebCam)





---
# PART 1: Setting up the environment


**STEP 1. Connect the Colab notebook to Google Drive**

We're gonna map your Google Drive folder. This first step is the only one that will require your manual interaction every time you run your notebook.

* Execute the following cell _(Click on Play button or press CTRL + ENTER)_ and click on the link to authorize your notebook to access to your Google Drive. 
* Paste the code Google will give to you and push `enter`

In [1]:
# This cell imports the drive library and mounts your Google Drive as a VM local drive. You can access to your Drive files 
# using this path "/content/drive/My Drive/"

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**STEP 2. Creating a new directory named MASKDETECTION and changing to that directory**

Run the following cells to create a folder named MASKDETECTION on your drive.

We then change the directory to this folder.

**NOTE:** You need to create the directory only once. So skip creating directory on further runs.

In [2]:
# This creates a new directory in your drive

%mkdir drive/My\ Drive/MASKDETECTION

In [3]:
#This makes the newly created directory your working directory

%cd drive/My\ Drive/MASKDETECTION

/content/drive/My Drive/MASKDETECTION


**STEP 3. Downloading dataset and required files**

Now we download the image dataset of with and without mask images into our folder. We also need weights and configuration file.

**NOTE:** Remember that you need to successfully download these files only once. You can skip this step on futher runs.

In [4]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1P9YgcPTZNufjC45YhtIJoGMz5Y4KtJfo' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1P9YgcPTZNufjC45YhtIJoGMz5Y4KtJfo" -O FILE && rm -rf /tmp/cookies.txt

--2023-04-30 18:29:38--  https://docs.google.com/uc?export=download&confirm=&id=1P9YgcPTZNufjC45YhtIJoGMz5Y4KtJfo
Resolving docs.google.com (docs.google.com)... 142.251.2.101, 142.251.2.102, 142.251.2.138, ...
Connecting to docs.google.com (docs.google.com)|142.251.2.101|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-04-30 18:29:38 ERROR 404: Not Found.



In [5]:
!unzip FILE

Archive:  FILE
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of FILE or
        FILE.zip, and cannot find FILE.ZIP, period.


**STEP 4. Importing required modules**

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import imutils
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
import cv2
from google.colab.patches import cv2_imshow
from imutils.video import VideoStream
import time
import sys

**Step 5. Importing the face detection model**

Since our main focus here is to detect masks we will not go into creating a face detection model. We simply import a pretrained model res10_300x300_ssd_iter_140000.caffemodel.

In [8]:
#Setting path to configuration file
prototxtPath = os.path.sep.join(["face_detector", "deploy.prototxt"])

#setting path to weights
weightsPath = os.path.sep.join(["face_detector",
	"res10_300x300_ssd_iter_140000.caffemodel"])

#Creating the network to detect faces
net = cv2.dnn.readNet(prototxtPath, weightsPath)

error: ignored







---





# PART 2: Training the model



**Step 1. Converting the images to array and storing their labels(with mask or withour mask)**

In [ ]:
imagePaths = list(paths.list_images("dataset"))
data = []
labels = []

# loop over the image paths
for imagePath in imagePaths:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]

	# load the input image (224x224) and preprocess it
	image = load_img(imagePath, target_size=(224, 224))
	image = img_to_array(image)
	image = preprocess_input(image)

	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)
 
data = np.array(data, dtype="float32")
labels = np.array(labels)

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [ ]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

**Step 2. Dividing the dataset for training and testing purposes**

In [ ]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)

**Step 3. Creating the model**

We will use MobileNetv2 as our base model and then apply further layers over it to create a final model.

In [ ]:
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

9412608/9406464 [==============================] - 0s 0us/step


Compiling our model

In [ ]:
LR = 1e-4     #learning rate
EPOCHS = 20    #no. of epochs
BS = 32       #batch size

#We will use Adam Optimizer for our model
opt= Adam(lr=LR, decay=LR / EPOCHS)

#To compile the model we use 'binary crossentropy' as loss function and set the metrics to accuracy
model.compile(loss="binary_crossentropy", optimizer=opt,
metrics=["accuracy"])

Construct the training image generator for data augmentation


In [ ]:
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

Training our model

In [ ]:
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

Make predictions on the testing set

In [ ]:
preds = model.evaluate(testX, testY)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

Serialize the model to disk

In [ ]:
#Here "model" is the name of model. You can change it accordingly.

model.save("model", save_format="h5")




---


# PART 3. Detection on Images

Let us load the model which we created.

In [ ]:
#This will load the pretrained model named 'mask_detector'. You can replace it with your model name.

model = load_model("mask_detector.model")

Importing the image for mask detection

In [ ]:
from google.colab import files
uploaded = files.upload() 
for name, data in uploaded.items():
  with open(name, 'wb') as f:
    f.write(data)
    print ('saved file', name)
    image = cv2.imread(name)
    orig = image.copy()
    (h, w) = image.shape[:2]


Saving smile.jpg to smile.jpg
saved file smile.jpg


Construct a blob from the image and then pass it through the imported face detector network to obtain face detections

In [ ]:
blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300),
	(104.0, 177.0, 123.0))

net.setInput(blob)
detections = net.forward()

Looping over the detections and labelling them as 'Mask' or 'No mask'

In [ ]:
for i in range(0, detections.shape[2]):
	# extract the confidence (i.e., probability) associated with
	# the detection
	confidence = detections[0, 0, i, 2]

	# filter. out weak detections by ensuring the confidence is
	# greater than the minimum confidence
	if confidence > 0.5:
		# compute the (x, y)-coordinates of the bounding box for
		# the object
		box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
		(startX, startY, endX, endY) = box.astype("int")

		# ensure the bounding boxes fall within the dimensions of
		# the frame
		(startX, startY) = (max(0, startX), max(0, startY))
		(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

		# extract the face ROI, convert it from BGR to RGB channel
		# ordering, resize it to 224x224, and preprocess it
		face = image[startY:endY, startX:endX]
		face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
		face = cv2.resize(face, (224, 224))
		face = img_to_array(face)
		face = preprocess_input(face)
		face = np.expand_dims(face, axis=0)

		# pass the face through the model to determine if the face
		# has a mask or not
		(mask, withoutMask) = model.predict(face)[0]

		# determine the class label and color we'll use to draw
		# the bounding box and text
		label = "Mask" if mask > withoutMask else "No Mask"
		color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

		# include the probability in the label
		label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

		# display the label and bounding box rectangle on the output
		# frame
		cv2.putText(image, label, (startX, startY - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)


Displaying the final image

In [ ]:
cv2_imshow(image)



---

# Part 4. Detection on videos (using WebCam)


A major challange which I faced was the real time detection on objects(masks).
Colab codes executes on a VM that doesn't have a webcam attached. So real time object detection is not possible here. But with the help of Javascript we can use the webcam to capture images and videos and use them for further actions. Though it is not real time but it works.

First of all we need to set the webcam with the following code. Your camera will start if it is working.
Clear the output afterwards to stop streaming.

In [ ]:
!pip install ffmpeg-python


from IPython.display import HTML, Javascript, display
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import io
import ffmpeg

video_file= '/content/drive/My Drive/MASKDETECTION/train.mp4' 

VIDEO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var my_btn_txt = document.createTextNode("Press to start recording");

my_btn.appendChild(my_btn_txt);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, videoStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  videoStream = stream;
  var options = {  
    mimeType : 'video/webm;codecs=vp9'  
  };            
  recorder = new MediaRecorder(stream, options);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('video');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({video: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      videoStream.getVideoTracks()[0].stop();
      recordButton.innerText = "Original recording"
  }
}

function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def start_webcam():
  js = Javascript('''
    async function startWebcam() {
      const div = document.createElement('div');

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      
      return;

    }
    ''')
  
  display(js)
  data = eval_js('startWebcam()')
  
start_webcam()

def get_video():
  display(HTML(VIDEO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  return binary


Creating a function to detect and predict masks 

In [ ]:
def detect_and_predict_mask(frame, faceNet, maskNet):
	# grab the dimensions of the frame and then construct a blob
	# from it
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),
		(104.0, 177.0, 123.0))

	# pass the blob through the network and obtain the face detections
	faceNet.setInput(blob)
	detections = faceNet.forward()

	# initialize our list of faces, their corresponding locations,
	# and the list of predictions from our face mask network
	faces = []
	locs = []
	preds = []

	# loop over the detections
	for i in range(0, detections.shape[2]):
		# extract the confidence (i.e., probability) associated with
		# the detection
		confidence = detections[0, 0, i, 2]

		# filter out weak detections by ensuring the confidence is
		# greater than the minimum confidence
		if confidence > 0.5:
			# compute the (x, y)-coordinates of the bounding box for
			# the object
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			# ensure the bounding boxes fall within the dimensions of
			# the frame
			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

			# extract the face ROI, convert it from BGR to RGB channel
			# ordering, resize it to 224x224, and preprocess it
			face = frame[startY:endY, startX:endX]
			face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
			face = cv2.resize(face, (224, 224))
			face = img_to_array(face)
			face = preprocess_input(face)

			# add the face and bounding boxes to their respective
			# lists
			faces.append(face)
			locs.append((startX, startY, endX, endY))

	# only make a predictions if at least one face was detected
	if len(faces) > 0:
		# for faster inference we'll make batch predictions on *all*
		# faces at the same time rather than one-by-one predictions
		# in the above `for` loop
		faces = np.array(faces, dtype="float32")
		preds = maskNet.predict(faces, batch_size=32)

	# return a 2-tuple of the face locations and their corresponding
	# locations
	return (locs, preds)

Load the model which we created.

In [ ]:
maskNet = load_model("mask_detector.model")

Capturing the video and detecting faces with or without mask

In [ ]:
vid = get_video()

with open(video_file, 'wb') as f:
  f.write(vid)
print("Recorded the video")


video = cv2.VideoCapture('train.mp4')
out = cv2.VideoWriter('out.avi', cv2.VideoWriter_fourcc(*'XVID'), 20.0, (640,480))
# print(type(out))


while True:
			# grab the frame from the threaded video stream and resize it
			# to have a maximum width of 400 pixels
			# frame = vs.read()
			# frame = imutils.resize(frame, width=400)
		boool, frame = video.read()
		
		if(boool==False):
			break
		w, h, c = frame.shape

		#syntax: cv2.resize(img, (width, height))
		frame = cv2.resize(frame,(h, w))

		# detect faces in the frame and determine if they are wearing a
		# face mask or not
		locs, preds = detect_and_predict_mask(frame, net, maskNet)
		# loop over the detected face locations and their corresponding
		# locations

	 
		for (box, pred) in zip(locs, preds):
					# unpack 	# frame = vs.read()
					# frame = imutils.resize(frame, width=400)the bounding box and predictions
					(startX, startY, endX, endY) = box
					(mask, withoutMask) = pred

					# determine the class label and color we'll use to draw
					# the bounding box and text
					label = "Mask" if mask > withoutMask else "No Mask"
					color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

					# include the probability in the label
					label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

					# display the label and bounding box rectangle on the output
					# frame
					cv2.putText(frame, label, (startX, startY - 10),
					cv2.FONT_HERSHEY_COMPLEX, 0.55, color, 2)
					cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

		# show the output frame
		out.write(frame)
		# cv2_imshow(frame)

out.release()
# do a bit of cleanup
cv2.destroyAllWindows()

print("Detection complete.Now converting")

!ffmpeg -i out.avi output.mp4 -loglevel quiet -y 

from IPython.display import HTML
from base64 import b64encode
mp4 = open('output.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)



---

# SOURCES


1.   chandrikadeb7/Face-Mask-Detection repo [github](https://github.com/chandrikadeb7/Face-Mask-Detection)
2.   [Tensorflow](https://www.tensorflow.org)







## **THANK YOU!**